In [1]:
from talib import abstract
import pandas as pd
import numpy as np
from talib.abstract import *
from quantfreedom._typing import (
    plFrame, plSeries, Union
)
df = pd.read_csv('30min.csv', index_col='time')

In [2]:
def testing(
    func_name: list,
    prices,
    **kwargs,
):
    np.empty()
    func_name = func_name.upper()
    talib_func = getattr(abstract, func_name)
    return talib_func(prices, **kwargs)

In [3]:
Function('rsi').info

{'name': 'RSI',
 'group': 'Momentum Indicators',
 'display_name': 'Relative Strength Index',
 'function_flags': ['Function has an unstable period'],
 'input_names': OrderedDict([('price', 'close')]),
 'parameters': OrderedDict([('timeperiod', 14)]),
 'output_flags': OrderedDict([('real', ['Line'])]),
 'output_names': ['real']}

In [4]:
Function('macd').info

{'name': 'MACD',
 'group': 'Momentum Indicators',
 'display_name': 'Moving Average Convergence/Divergence',
 'function_flags': None,
 'input_names': OrderedDict([('price', 'close')]),
 'parameters': OrderedDict([('fastperiod', 12),
              ('slowperiod', 26),
              ('signalperiod', 9)]),
 'output_flags': OrderedDict([('macd', ['Line']),
              ('macdsignal', ['Dashed Line']),
              ('macdhist', ['Histogram'])]),
 'output_names': ['macd', 'macdsignal', 'macdhist']}

In [5]:
Function('stoch').info

{'name': 'STOCH',
 'group': 'Momentum Indicators',
 'display_name': 'Stochastic',
 'function_flags': None,
 'input_names': OrderedDict([('prices', ['high', 'low', 'close'])]),
 'parameters': OrderedDict([('fastk_period', 5),
              ('slowk_period', 3),
              ('slowk_matype', 0),
              ('slowd_period', 3),
              ('slowd_matype', 0)]),
 'output_flags': OrderedDict([('slowk', ['Dashed Line']),
              ('slowd', ['Dashed Line'])]),
 'output_names': ['slowk', 'slowd']}